In [1]:
import numpy as np
import SimpleITK as sitk
import skimage
import cv2
import os, csv
import matplotlib.pyplot as plt

from matplotlib.patches import Rectangle


In [2]:
print(sitk.Version())


SimpleITK Version: 2.0.0rc2.dev909-g29cc1 (ITK 5.1)
Compiled: Mar  5 2021 06:23:01



In [2]:
control_path = '/data/scratch/aalexopoulos/OAI_IW-TSE_control_flipped/'
control_imgs = sorted(os.path.join(control_path, fname) for fname in os.listdir(control_path))

control_mask_path = '/data/scratch/aalexopoulos/OAI_IW-TSE_control_seg_flipped/'
control_mask = sorted(os.path.join(control_mask_path, fname) for fname in os.listdir(control_mask_path))

In [4]:
fb_xmin_c_global_xcoord = 33
fb_xmin_c_global_ycoord = 133
fb_xmax_c_global_xcoord = 339
fb_xmax_c_global_ycoord = 209

tb_xmin_c_global_xcoord = 43
tb_xmin_c_global_ycoord = 197
tb_xmax_c_global_xcoord = 320
tb_xmax_c_global_ycoord = 288

tb_ymin_c_global_xcoord = 153
tb_ymin_c_global_ycoord = 169

rect_xmin_global = 33
rect_ymin_global = 133 
rect_width_orig = fb_xmax_c_global_xcoord-fb_xmin_c_global_xcoord
rect_height_orig = tb_xmax_c_global_ycoord-fb_xmin_c_global_ycoord
rect_height = 250
rect_height_230 = 230
rect_width = 320
print('Rect Height {} and Width {} based on the bone min max coords'.format(rect_height_orig, rect_width_orig))
print('Rect Height {} and Width {} to incorporate as many pixels as possible'.format(rect_height, rect_width))

Rect Height 155 and Width 306 based on the bone min max coords
Rect Height 250 and Width 320 to incorporate as many pixels as possible


In [9]:
count = 0
for j in range(0,len(control_imgs[:100])):
    input_img = sitk.ReadImage(control_imgs[j])
    input_img_np = sitk.GetArrayFromImage(input_img)

    input_mask = sitk.ReadImage(control_mask[j])
    input_mask_np = sitk.GetArrayFromImage(input_mask)
    import pathlib
    main_path = '/data/scratch/aalexopoulos/crop_ctr/'+control_imgs[j].split('/')[-1].split('.')[0]+'/'
    p_compare = pathlib.Path(main_path+'compare/')
    p_compare.mkdir(parents=True, exist_ok=True)
    print(str(p_compare))

    p_crop_250_320_lat = pathlib.Path(main_path+'lateral/')
    p_crop_250_320_lat.mkdir(parents=True,exist_ok=True)
    print(str(p_crop_250_320_lat))

    p_crop_250_320_med = pathlib.Path(main_path+'medial/')
    p_crop_250_320_med.mkdir(parents=True,exist_ok=True)
    print(str(p_crop_250_320_med))

    p_crop_250_320 = pathlib.Path(main_path+'whole_knee/')
    p_crop_250_320.mkdir(parents=True,exist_ok=True)
    print(str(p_crop_250_320))

    for i in range(input_mask_np.shape[0]):
        knee_mask = input_mask_np[i,:,:].copy()
        knee_canny = skimage.feature.canny(knee_mask,sigma=8,low_threshold=0.05,high_threshold=0.4)
        knee_canny_true_count = np.count_nonzero(knee_canny)
        print('Whole Knee Canny Edge ONES: {}, Whole Knee Bone Pixels: {}'.format(knee_canny_true_count,len(np.where(input_mask_np[i,:,:]>0.0)[0])))
        
        if len(np.where(input_mask_np[i,:,:]>0.0)[0])>500:

            fig,((ax4,ax5,ax6),(ax7,ax8,ax9)) = plt.subplots(nrows=2,ncols=3,figsize=(20,12)
                                                ,sharex=True,sharey=True)

            ax4.add_patch(Rectangle((rect_xmin_global,rect_ymin_global),rect_width,rect_height,fill=False,edgecolor='w'))
            ax4.plot(rect_xmin_global+int(rect_width/2), tb_ymin_c_global_ycoord, marker="o", markersize=5, markerfacecolor='red')
            ax4.imshow(input_img_np[i,:,:],cmap=plt.cm.gray)
            ax4.set_title('IW-TSE MRI 250-320_Bottom',fontsize=20)
            ax4.set_aspect('equal')

            ax5.add_patch(Rectangle((rect_xmin_global,(rect_ymin_global-20)),rect_width,rect_height,fill=False,edgecolor='w'))
            ax5.plot(rect_xmin_global+int(rect_width/2), tb_ymin_c_global_ycoord, marker="o", markersize=5, markerfacecolor='red')
            ax5.imshow(input_img_np[i,:,:],cmap=plt.cm.gray)
            ax5.set_title('IW-TSE MRI 250-320_mid',fontsize=20)
            ax5.set_aspect('equal')

            ax6.add_patch(Rectangle((rect_xmin_global,(rect_ymin_global-40)),rect_width,rect_height,fill=False,edgecolor='w'))
            ax6.plot(rect_xmin_global+int(rect_width/2), tb_ymin_c_global_ycoord, marker="o", markersize=5, markerfacecolor='red')
            ax6.imshow(input_img_np[i,:,:],cmap=plt.cm.gray)
            ax6.set_title('IW-TSE MRI 250-320_Top',fontsize=20)
            ax6.set_aspect('equal')

            ax7.plot([tb_xmin_c_global_xcoord,tb_xmax_c_global_xcoord],[tb_xmin_c_global_ycoord,tb_xmax_c_global_ycoord],'y',lw=2)
            ax7.plot([fb_xmin_c_global_xcoord,fb_xmax_c_global_xcoord],[fb_xmin_c_global_ycoord,fb_xmax_c_global_ycoord],'m',lw=2)
            ax7.plot(rect_xmin_global+int(rect_width/2), tb_ymin_c_global_ycoord, marker="o", markersize=5, markerfacecolor='red')
            ax7.imshow(input_mask_np[i,:,:],cmap=plt.cm.gray)
            ax7.set_title('IW-TSE MASK',fontsize=20)
            ax7.set_aspect('equal')

            fb_mask = input_mask_np[i,:,:].copy()
            fb_mask[np.where(fb_mask!=1.0)]=0.0
            fb_canny = skimage.feature.canny(fb_mask,sigma=8,low_threshold=0.37,high_threshold=0.39)
            ax8.plot([tb_xmin_c_global_xcoord,tb_xmax_c_global_xcoord],[tb_xmin_c_global_ycoord,tb_xmax_c_global_ycoord],'y',lw=2)
            ax8.plot([fb_xmin_c_global_xcoord,fb_xmax_c_global_xcoord],[fb_xmin_c_global_ycoord,fb_xmax_c_global_ycoord],'m',lw=2)
            ax8.imshow(fb_canny,cmap=plt.cm.gray)
            ax8.set_title('FEMURAL CANNY EDGE',fontsize=20)
            ax8.set_aspect('equal')
            
            tb_mask = input_mask_np[i,:,:].copy()
            tb_mask[np.where(tb_mask!=3.0)]=0.0
            tb_canny = skimage.feature.canny(tb_mask,sigma=8,low_threshold=0.9,high_threshold=0.95)
            ax9.plot([tb_xmin_c_global_xcoord,tb_xmax_c_global_xcoord],[tb_xmin_c_global_ycoord,tb_xmax_c_global_ycoord],'y',lw=2)
            ax9.plot([fb_xmin_c_global_xcoord,fb_xmax_c_global_xcoord],[fb_xmin_c_global_ycoord,fb_xmax_c_global_ycoord],'m',lw=2)
            ax9.imshow(tb_canny,cmap=plt.cm.gray)
            ax9.set_title('TIBIAL CANNY EDGE',fontsize=20)
            ax9.set_aspect('equal')
            

            savepath1 = str(p_compare)+'/'+control_imgs[j].split('/')[-1].split('.')[0]+'_slice_'+str(i)+'_compare.png'
            plt.savefig(savepath1,bbox_inches='tight',dpi=100,facecolor='white')
            plt.close()

            savepath2 = str(p_crop_250_320_lat)+'/'+control_imgs[j].split('/')[-1].split('.')[0]+'_slice_'+str(i)+'_lat.png'
            crop_img = input_img_np[i,rect_ymin_global:rect_ymin_global+rect_height,rect_xmin_global:rect_xmin_global+int(rect_width/2)]
            crop_img_norm = (crop_img/crop_img.max())*255
            cv2.imwrite(savepath2,crop_img_norm,[cv2.IMWRITE_WEBP_QUALITY,100])

            savepath3 = str(p_crop_250_320_med)+'/'+control_imgs[j].split('/')[-1].split('.')[0]+'_slice_'+str(i)+'_med.png'
            crop_img = input_img_np[i,(rect_ymin_global-20):rect_ymin_global+rect_height,rect_xmin_global+int(rect_width/2):rect_xmin_global+rect_width]
            crop_img_norm = (crop_img/crop_img.max())*255
            cv2.imwrite(savepath3,crop_img_norm,[cv2.IMWRITE_WEBP_QUALITY,100])

            savepath4 = str(p_crop_250_320)+'/'+control_imgs[j].split('/')[-1].split('.')[0]+'_slice_'+str(i)+'.png'
            crop_img = input_img_np[i,(rect_ymin_global-40):rect_ymin_global+rect_height,rect_xmin_global:rect_xmin_global+rect_width]
            crop_img_norm = (crop_img/crop_img.max())*255
            cv2.imwrite(savepath4,crop_img_norm,[cv2.IMWRITE_WEBP_QUALITY,100])



/media/anastasis/My Harddisk2/ThesisImages/canny_edges_control_results/9242547_IW-TSE_R/compare
/media/anastasis/My Harddisk2/ThesisImages/canny_edges_control_results/9242547_IW-TSE_R/lateral
/media/anastasis/My Harddisk2/ThesisImages/canny_edges_control_results/9242547_IW-TSE_R/medial
/media/anastasis/My Harddisk2/ThesisImages/canny_edges_control_results/9242547_IW-TSE_R/whole_knee
Whole Knee Canny Edge ONES: 0, Whole Knee Bone Pixels: 0
Whole Knee Canny Edge ONES: 0, Whole Knee Bone Pixels: 0
Whole Knee Canny Edge ONES: 0, Whole Knee Bone Pixels: 0
Whole Knee Canny Edge ONES: 0, Whole Knee Bone Pixels: 0
Whole Knee Canny Edge ONES: 0, Whole Knee Bone Pixels: 0
Whole Knee Canny Edge ONES: 0, Whole Knee Bone Pixels: 0
Whole Knee Canny Edge ONES: 0, Whole Knee Bone Pixels: 0
Whole Knee Canny Edge ONES: 92, Whole Knee Bone Pixels: 663
Whole Knee Canny Edge ONES: 234, Whole Knee Bone Pixels: 3282
Whole Knee Canny Edge ONES: 369, Whole Knee Bone Pixels: 7594
Whole Knee Canny Edge ONES: 789